# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from iextoken import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data, '\n', api_url)

{'companyName': 'Apple Inc', 'marketcap': 2743651373680, 'week52high': 197.7, 'week52low': 123.48, 'week52highSplitAdjustOnly': 198.23, 'week52lowSplitAdjustOnly': 124.17, 'week52change': 0.15139585998753424, 'sharesOutstanding': 15634232000, 'float': 0, 'avg10Volume': 74616290, 'avg30Volume': 65485572, 'day200MovingAvg': 175.41, 'day50MovingAvg': 179.55, 'employees': 164000, 'ttmEPS': 5.95, 'ttmDividendRate': 0.9368421399305272, 'dividendYield': 0.005338436035845501, 'nextDividendDate': '', 'exDividendDate': '2023-08-11', 'nextEarningsDate': '2023-11-02', 'peRatio': 28.95368693203884, 'beta': 1.3257803316536951, 'maxChangePercent': 67.4118197411508, 'year5ChangePercent': 2.3397912654248154, 'year2ChangePercent': 0.24516804695437955, 'year1ChangePercent': 0.12805975521958235, 'ytdChangePercent': 0.3582438879412031, 'month6ChangePercent': 0.11948986628510404, 'month3ChangePercent': -0.0488934559057056, 'month1ChangePercent': 0.005730987449137581, 'day30ChangePercent': -0.001990445859872

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
market_cap = data['marketcap']
price = data['day200MovingAvg']


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)


In [7]:
final_dataframe.append(
    pd.Series([symbol, price, market_cap, 'N/A'], index = my_columns),
    ignore_index = True)

C:\Users\user\AppData\Local\Temp\ipykernel_34512\442051218.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,175.41,2743651373680,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks["Ticker"]:
    try:
        api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/stats/?token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
        final_dataframe = final_dataframe.append(
            pd.Series([stock, data["day200MovingAvg"], data["marketcap"], "N/A"], index = my_columns),
            ignore_index = True
    )
    except:
        print(f'{stock} is not found')
        pass

C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\

AET is not found


C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


GAS is not found


C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


...

C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\

YHOO is not found


C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


ZMH is not found


C:\Users\user\AppData\Local\Temp\ipykernel_34512\4215612419.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,100.93,54641730646,N/A
1,ABT,104.76,174021698234,N/A
2,ANF,34.20,2530442818,N/A
3,ACN,295.93,210453956698,N/A
4,ADBE,446.45,244208524000,N/A
...,...,...,...,...
419,XLNX,0.00,0,N/A
420,XL,0.00,181592034,N/A
421,XYL,103.97,22534325365,N/A
422,YUM,132.37,35600843251,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [12]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [27]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    print(data)
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe.append(
                pd.Series([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['stats']['marketcap'], 'N/A'], index = my_columns),
                ignore_index = True
            )
        except:
            print(f'{symbol} is not found')
            pass

final_dataframe

{'MMM': {'stats': {'companyName': '3M Co.', 'marketcap': 54641730646, 'week52high': 126.78, 'week52low': 91.02, 'week52highSplitAdjustOnly': 133.91, 'week52lowSplitAdjustOnly': 92.38, 'week52change': -0.08929323856119564, 'sharesOutstanding': 551992430, 'float': 0, 'avg10Volume': 4340531, 'avg30Volume': 4005563, 'day200MovingAvg': 100.93, 'day50MovingAvg': 102.71, 'employees': 48055, 'ttmEPS': -2.63, 'ttmDividendRate': 5.860834243365432, 'dividendYield': 0.05920632632958311, 'nextDividendDate': '', 'exDividendDate': '2023-08-18', 'nextEarningsDate': '2023-10-24', 'peRatio': -37.27266756186904, 'beta': 0.9853699378210878, 'maxChangePercent': 0.9999878776368664, 'year5ChangePercent': -0.4426219908400592, 'year2ChangePercent': -0.3983998249713604, 'year1ChangePercent': -0.1026581226255432, 'ytdChangePercent': -0.13819146444767738, 'month6ChangePercent': -0.022168397636371902, 'month3ChangePercent': -0.017884249438946842, 'month1ChangePercent': -0.01019898010198994, 'day30ChangePercent': -

C:\Users\user\AppData\Local\Temp\ipykernel_34512\4196259455.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4196259455.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4196259455.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_34512\4196259455.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Lo

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,97.565,54641730646,N/A
1,ABT,99.030,174021698234,N/A
2,ANF,49.460,2530442818,N/A
3,ACN,312.100,210453956698,N/A
4,ADBE,514.400,244208524000,N/A
...,...,...,...,...
80,CHK,82.835,11092791812,N/A
81,CVX,165.225,311251105388,N/A
82,CMG,1860.310,52981459014,N/A
83,CB,212.960,87798707615,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [30]:
portfolio_size = input('Enter the value of your portfolio:')
try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter an integer')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

In [33]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe


# number_of_apple_shares = position_size/424
# print(math.floor(number_of_apple_shares)) # If fractional shares not allowed



,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,97.565,54641730646,1205
1,ABT,99.030,174021698234,1187
2,ANF,49.460,2530442818,2378
3,ACN,312.100,210453956698,376
4,ADBE,514.400,244208524000,228
...,...,...,...,...
80,CHK,82.835,11092791812,1420
81,CVX,165.225,311251105388,712
82,CMG,1860.310,52981459014,63
83,CB,212.960,87798707615,552


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [46]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [47]:
background_color = '#0a0a23'
fontcolor = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': fontcolor,
        'bg_color': background_color,
        'border': 1
    }
)
    
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': fontcolor,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': fontcolor,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [42]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)

# writer.save()

# writer.sheets['Recommended Trades'].write['A1', 'Ticker', string_format]
# writer.sheets['Recommended Trades'].write['B1', 'Stock Price', dollar_format]
# writer.sheets['Recommended Trades'].write['C1', 'Market Capitalisation', dollar_format]
# writer.sheets['Recommended Trades'].write['D1', 'Ticker', integer_format]

C:\Users\user\AppData\Local\Temp\ipykernel_34512\3580212437.py:6: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [48]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [49]:
writer.save()

C:\Users\user\AppData\Local\Temp\ipykernel_34512\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
